In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [5]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [6]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [7]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [8]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [9]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [10]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [11]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [12]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [13]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [14]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//4).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

8255667


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [15]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [16]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

14860201
14852772


In [17]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad
14054628,1112496031,rmdwdxb,0,FEM,19.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,2.0,NaN,NaN
8439511,1112257940,Z93AK5,1,FEM,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,33.0,NaN,NaN
13084559,1112274204,Drp0dBL,0,FEM,24.0,-,-,-,-,Graduado,...,True,False,True,True,False,False,False,1.0,NaN,NaN
7295174,1112273596,NKx94z,1,MASC,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
832060,1112296354,xkaa49j,1,MASC,39.0,-,-,-,-,Graduado,...,False,False,True,True,False,False,False,25.0,1.0,0.040000
8353429,1112445305,EX3xW9,0,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,True,6.0,NaN,NaN
12826355,1112390296,6r2zwex,0,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
715593,1112393208,969jq3a,0,FEM,26.0,-,-,-,-,Graduado,...,True,False,True,True,False,True,False,43.0,NaN,NaN
8973755,1112379097,8MPezox,1,NO_DECLARA,200.0,-,-,-,-,En Curso,...,True,True,False,False,False,False,False,68.0,NaN,NaN
6058740,1112322830,ZDr3zz5,0,FEM,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,2.0,NaN,NaN


In [18]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
10548260,1112447001,8MraRZx,0,FEM,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,2.0,0.0,0.000000,False
187218,1112377249,lDw603R,1,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,9.0,0.0,0.000000,False
12850041,1112397590,zvxjEz3,0,MASC,19.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,7.0,0.0,0.000000,False
427734,1112260584,ow2EXqW,1,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,True,False,True,0.0,0.0,0.000000,False
11492214,1491320,xkP1mWx,0,MASC,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,22.0,0.0,0.000000,False
2751990,1112392372,5mqojpq,0,MASC,52.0,-,-,-,-,Graduado,...,False,False,True,True,False,True,2.0,0.0,0.000000,False
12805449,1112443820,NzrGbNB,0,MASC,42.0,-,-,-,-,-,...,False,False,False,False,False,False,3.0,0.0,0.000000,False
1556385,1112359843,PmGwG1a,1,MASC,24.0,-,-,-,-,-,...,False,False,False,False,False,False,10.0,2.0,0.200000,True
2493888,1112293412,8MBZN6j,1,MASC,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,0.0,0.0,0.000000,False
4093771,1112283554,jkjvkzB,1,FEM,21.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,0.0,0.0,0.000000,False


In [19]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,1
2,0z5Dmrd,Almacén / Depósito / Expedición,1
3,0z5Dmrd,Atención al Cliente,1
4,0z5Dmrd,Auditoría,1


In [20]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
1,0z5Dmrd,Junior,2
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,14
4,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2


In [21]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,17
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,24
3,0z5JW1r,Part-time,1
4,0z5JW1r,Teletrabajo,1


In [22]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
7042184,1112442348,8JY4BR,1,FEM,42.0,-,-,-,-,-,...,233.0,2.0,0.008584,True,24,14,36,110.0,81.0,81.0
9043836,1111926163,A3RNlzl,0,FEM,20.0,-,-,-,-,-,...,12.0,0.0,0.000000,False,1,14,4,4.0,6.0,6.0
4502691,1111959242,lDLZQW2,1,MASC,41.0,-,-,-,-,Graduado,...,35.0,1.0,0.028571,True,3,0,6,4.0,1.0,1.0
12916165,1112410690,NzPV8Px,0,FEM,33.0,-,-,-,-,Graduado,...,1.0,0.0,0.000000,False,3,11,9,0.0,0.0,0.0
3353119,1112463443,Ejb554,1,MASC,28.0,-,-,Graduado,-,Graduado,...,1553.0,1.0,0.000644,True,13,1488,1071,8.0,1186.0,1186.0
1548435,1112327106,eoO8rW,1,MASC,39.0,-,-,-,-,-,...,37.0,0.0,0.000000,False,21,77,20,10.0,25.0,25.0
10213234,1112470631,VNrw0kj,0,FEM,26.0,-,-,-,-,-,...,72.0,0.0,0.000000,False,0,22,12,0.0,69.0,69.0
2857701,1112410555,KBvY8Jj,1,MASC,28.0,-,-,Graduado,-,Graduado,...,21.0,7.0,0.333333,True,0,14,12,12.0,21.0,21.0
11606429,1112152855,akjB4RJ,0,FEM,25.0,-,-,-,-,-,...,2.0,0.0,0.000000,False,0,14,11,0.0,0.0,0.0
11521109,1112480127,EzEvmJ9,0,FEM,23.0,-,-,-,-,Graduado,...,8.0,0.0,0.000000,False,1,16,1,0.0,2.0,2.0


In [23]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  13715101 Test:  721848


In [24]:
gc.collect()

312

In [25]:
c=lgb.LGBMClassifier(learning_rate=0.05,objective='binary',num_leaves=12000,max_depth=18,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.65893
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.62792
[3]	valid_0's binary_logloss: 0.598668
[4]	valid_0's binary_logloss: 0.570931
[5]	valid_0's binary_logloss: 0.545758
[6]	valid_0's binary_logloss: 0.523402
[7]	valid_0's binary_logloss: 0.502651
[8]	valid_0's binary_logloss: 0.48264
[9]	valid_0's binary_logloss: 0.464919
[10]	valid_0's binary_logloss: 0.448535
[11]	valid_0's binary_logloss: 0.43244
[12]	valid_0's binary_logloss: 0.417502
[13]	valid_0's binary_logloss: 0.403706
[14]	valid_0's binary_logloss: 0.392306
[15]	valid_0's binary_logloss: 0.381196
[16]	valid_0's binary_logloss: 0.371401
[17]	valid_0's binary_logloss: 0.360608
[18]	valid_0's binary_logloss: 0.351581
[19]	valid_0's binary_logloss: 0.341987
[20]	valid_0's binary_logloss: 0.334051
[21]	valid_0's binary_logloss: 0.325651
[22]	valid_0's binary_logloss: 0.317704
[23]	valid_0's binary_logloss: 0.311103
[24]	valid_0's binary_logloss: 0

In [26]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9838807544482505


In [35]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV29.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV29.pkl']

In [27]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['lo_vio']=(prediccion['cantidad']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
7823,89618,1112467034,GNX3b8v,MASC,21.0,-,-,-,-,Graduado,...,216.0,1.0,0.004630,True,17.0,36.0,47.0,17.0,45.0,45.0
12852,46566,1112430193,1QraqqR,FEM,26.0,-,-,Graduado,-,Graduado,...,27.0,2.0,0.074074,True,1.0,20.0,17.0,5.0,12.0,12.0
39282,83816,1112466410,4rPY4N5,FEM,21.0,-,-,-,-,Graduado,...,183.0,1.0,0.005464,True,48.0,103.0,175.0,33.0,80.0,80.0
22684,18392,1112213212,kPjDdvm,FEM,21.0,-,-,-,-,Graduado,...,4.0,NaN,0.000000,False,0.0,36.0,0.0,0.0,4.0,4.0
12208,54749,1112446968,56XeZX,FEM,46.0,En Curso,-,-,Graduado,Graduado,...,6.0,NaN,0.000000,False,0.0,15.0,0.0,0.0,4.0,4.0
63411,10796,1111926843,e4rxlr,FEM,37.0,-,-,Graduado,-,Graduado,...,1.0,NaN,0.000000,False,0.0,17.0,13.0,0.0,1.0,1.0
90581,91966,1112473393,8KljOO,MASC,42.0,-,-,-,-,Graduado,...,8.0,NaN,0.000000,False,0.0,16.0,1.0,0.0,8.0,8.0
57535,69151,1112461125,8PL3KR,FEM,33.0,-,-,-,-,-,...,54.0,NaN,0.000000,False,1.0,34.0,25.0,0.0,42.0,42.0
99675,99639,1112513109,NmZM0V,FEM,45.0,-,-,-,-,-,...,NaN,NaN,0.000000,False,0.0,8.0,0.0,0.0,0.0,0.0
79936,27396,1112350157,Y0m81J,FEM,44.0,-,En Curso,-,-,Graduado,...,NaN,NaN,0.000000,False,0.0,13.0,10.0,0.0,0.0,0.0


In [28]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[9.92020452e-01, 7.97954761e-03],
       [9.91404663e-01, 8.59533741e-03],
       [9.98815507e-01, 1.18449256e-03],
       ...,
       [9.98491955e-01, 1.50804533e-03],
       [9.98925622e-01, 1.07437792e-03],
       [9.99580496e-01, 4.19503931e-04]])

In [29]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [30]:
prediccion['sepostulo']=y_final2

In [31]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,NaN,0.0,False,0.0,17.0,0.0,4.0,24.0,24.0,0.007980
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,NaN,0.0,False,5.0,58.0,0.0,0.0,6.0,6.0,0.008595
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,NaN,0.0,False,2.0,11.0,0.0,1.0,7.0,7.0,0.001184
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,NaN,0.0,False,0.0,14.0,0.0,0.0,0.0,0.0,0.000408
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,NaN,0.0,False,0.0,11.0,0.0,0.0,3.0,3.0,0.000084


In [32]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.007980
1,1,0.008595
2,2,0.001184
3,3,0.000408
4,4,0.000084


In [33]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion60.csv",index=False)

In [34]:
prediccion.sepostulo.mean()

0.4618903067183211